In [ ]:
!pip3 install keras-visualizer
!pip3 install livelossplot
# !pip3 install matplotlib --upgrade

# New Section

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot  as plt

from google.colab import drive
from collections import Counter

import pickle
import joblib

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.metrics import confusion_matrix,plot_confusion_matrix,accuracy_score,r2_score,classification_report,mean_squared_error
from math import sqrt

In [ ]:
drive.mount('/content/drive')

In [ ]:
ds = '/content/drive/MyDrive/project/dataset/dataset.csv'
ds = pd.read_csv(ds)
ds.head()

In [ ]:
ds.rename(columns = {'hours.per.week':'workHour'}, inplace = True)
ds.rename(columns = {'native.country':'Country'}, inplace = True)

In [ ]:
ds.info()

Take required COLUMNS

In [ ]:
headers = ['age','workclass','education','occupation','sex','workHour','Country','income']
dataframe  = pd.DataFrame(ds,columns=headers)
dataframe.head()

In [ ]:
dataframe.info()

Fix Missing Value

In [ ]:
imputer = SimpleImputer(missing_values='?', strategy='most_frequent')
imputer = imputer.fit(dataframe)
fdata = imputer.transform(dataframe)

df = pd.DataFrame(fdata,columns=headers).astype(dataframe.dtypes.to_dict())
df.head()

Plot Each Column Data

In [ ]:
fig,axs = plt.subplots(figsize=(20,10))
ax1 = sns.countplot(df['workclass'],ax=axs)
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45)


In [ ]:
fig,axs = plt.subplots(figsize=(20,10))
ax2 = sns.countplot(df['education'],ax=axs)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45)


In [ ]:
fig,axs = plt.subplots(figsize=(20,10))
ax3 = sns.countplot(df['occupation'],ax=axs)
ax3.set_xticklabels(ax3.get_xticklabels(), rotation=45)


In [ ]:
fig,axs = plt.subplots(figsize=(20,10))
ax4 = sns.countplot(df['sex'],ax=axs)
ax4.set_xticklabels(ax4.get_xticklabels(), rotation=45)


In [ ]:
fig,axs = plt.subplots(figsize=(20,10))
ax5 = sns.countplot(df['Country'],ax=axs)
ax5.set_xticklabels(ax5.get_xticklabels(), rotation=45)


In [ ]:
fig,axs = plt.subplots(figsize=(20,10))
ax6 = sns.countplot(df['income'],ax=axs)
ax6.set_xticklabels(ax6.get_xticklabels(), rotation=45)

check for data Bias 

In [ ]:
df['income'].value_counts().plot(kind='bar',figsize=(10,6))

target = df.values[:,-1]
counter = Counter(target)
for k,v in counter.items():
 per = v / len(target) * 100
 print('Items=%s, Count=%d, Percentage=%.3f%%' % (k, v, per))

In [ ]:
df['Country'].value_counts().plot(kind='bar',figsize=(10,6))

target = df.values[:,-1]
counter = Counter(target)
for k,v in counter.items():
 per = v / len(target) * 100
 print('Items=%s, Count=%d, Percentage=%.3f%%' % (k, v, per))

In [ ]:
df['sex'].value_counts().plot(kind='bar',figsize=(10,6))

target = df.values[:,-1]
counter = Counter(target)
for k,v in counter.items():
 per = v / len(target) * 100
 print('Items=%s, Count=%d, Percentage=%.3f%%' % (k, v, per))

FIX Bias By populating data from existing dataset using base of income

In [ ]:
data_sel =  df[df['income'] == '>50K']
new_df = pd.concat([data_sel] * 3,ignore_index=True)
df = new_df.append(df[df['income'] == '<=50K'],ignore_index=True)

In [ ]:
plt.subplots(figsize=(10,6))
ax = sns.countplot(df['income'])

target = df.values[:,-1]
counter = Counter(target)
for k,v in counter.items():
 per = v / len(target) * 100
 print('Items=%s, Count=%d, Percentage=%.3f%%' % (k, v, per))

 ENCODEcategorical to Numerical

In [ ]:
le = LabelEncoder()

# lebel encode all data which has datatype as object
for columns in df.columns[df.dtypes == 'object']:
  df[columns] = le.fit_transform(df[columns])


In [ ]:
df.head()

In [ ]:
# df.info()
# save encoder
# pickle.dump(le, open("/content/drive/MyDrive/Lbelencode.pkl", "wb"))

DATA COREALATION

In [ ]:
plt.figure(figsize = (16,10))
plt.title("Correlation between different features of the dataset", fontsize = 18, fontweight = 'bold')
sns.heatmap(df.corr(), cmap = 'Greens_r', annot = True)
plt.xticks(fontsize=12, rotation = 90)
plt.yticks(fontsize=12, rotation = 90)
plt.legend(fontsize=12)

SCALE DATA

In [ ]:
scaler = MinMaxScaler()
scaleddataset = scaler.fit_transform(df)
sdataset = pd.DataFrame(scaleddataset,columns=headers)
sdataset.head()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(26, 15))

ax1.set_title('Before Scaling')
sns.kdeplot(df['age'], ax=ax1)
sns.kdeplot(df['workclass'], ax=ax1)
sns.kdeplot(df['education'], ax=ax1)
sns.kdeplot(df['occupation'], ax=ax1)
sns.kdeplot(df['sex'], ax=ax1)
sns.kdeplot(df['workHour'], ax=ax1)
sns.kdeplot(df['Country'], ax=ax1)
sns.kdeplot(df['income'], ax=ax1)

ax2.set_title('After Standard Scaler')
sns.kdeplot(sdataset['age'], ax=ax2)
sns.kdeplot(sdataset['workclass'], ax=ax2)
sns.kdeplot(sdataset['education'], ax=ax2)
sns.kdeplot(sdataset['occupation'], ax=ax2)
sns.kdeplot(sdataset['sex'], ax=ax2)
sns.kdeplot(sdataset['workHour'], ax=ax2)
sns.kdeplot(sdataset['Country'], ax=ax2)
sns.kdeplot(sdataset['income'], ax=ax2)
plt.show()

In [ ]:
# save Scaler
# pickle.dump(le, open("/content/drive/MyDrive/Scaler.pkl", "wb")) 

In [ ]:
# pickle.dump(scaler, open("/content/drive/MyDrive/project/model/minmax.pkl", "wb"))

Splitting Data to TRAIN MODEL

In [ ]:
x_data = np.array(df.drop(columns=['income']))
y_data = np.array(df['income'])

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x_data,y_data,train_size=0.33,random_state=33)

In [ ]:
print("xtrain: ", x_train.shape)
print("xtest: ", x_test.shape)
print("ytrain: ", y_train.shape)
print("ytest: ", y_test.shape)


In [ ]:
print("xtrain: ", x_train)
print("xtest: ", x_test)
print("ytrain: ", y_train)
print("ytest: ", y_test)



```
r2_score is used in regression problems, whereas accuracy function is used in classification problem.
```

In [ ]:
from sklearn.model_selection import GridSearchCV

**KNN**

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knnparameter = {
    'n_neighbors': [3,5,7],
    'weights' : ['uniform','distance']
}

knn = KNeighborsRegressor()

grid_search = GridSearchCV(knn,param_grid=knnparameter,cv=7)
grid_search.fit(x_train,y_train)
print(grid_search.best_params_)

In [ ]:
knn_result=pd.DataFrame(grid_search.cv_results_)
knn_resulthm= knn_result[['param_n_neighbors' ,'param_weights','mean_test_score']]
knn_resulthm= knn_resulthm.pivot('param_n_neighbors' ,'param_weights','mean_test_score')
sns.heatmap(knn_resulthm,annot=True,fmt='.3f')

In [ ]:
sns.lineplot(data=knn_result, x='param_n_neighbors', y='mean_test_score', hue='param_weights')
plt.show()

In [ ]:
knny_pred = grid_search.predict(x_test)
print("R2 score of model Evaluation:  {:.2f}%" .format(r2_score(y_test,knny_pred)*100))
knnaccuracy = grid_search.best_score_ *100
print("Accuracy for our training dataset with tuning is : {:.2f}%".format(knnaccuracy) )


In [ ]:
knny_pred = np.where(knny_pred>0.5,1,0)
print("ACcuracy Score:",accuracy_score(y_test,knny_pred)*100)


In [ ]:
rmse = sqrt(mean_squared_error(y_test, knny_pred))
print("Root Mean Squared Error: ", rmse*100)

**Linear Regression**


In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
linparam = {
    'fit_intercept': [True, False],
    'normalize': [True, False]
}
linmodel = LinearRegression()

linGrid_search = GridSearchCV(linmodel,linparam,cv=5)
linGrid_search.fit(x_train, y_train)

In [ ]:
liny_pred = linGrid_search.predict(x_test)
print("R2 score of model Evaluation:  {:.2f}%" .format(r2_score(y_test,liny_pred)*100))
linaccuracy = linGrid_search.best_score_ *100
print("Accuracy for our training dataset with tuning is : {:.2f}%".format(linaccuracy) )


In [ ]:
liny_pred = np.where(liny_pred>0.5,1,0)
print("ACcuracy Score:",accuracy_score(y_test,liny_pred)*100)


In [ ]:
lrrmse = sqrt(mean_squared_error(y_test, liny_pred))
print("Root Mean Squared Error: ", lrrmse*100)

**XGBOOST**

In [ ]:
import xgboost as xgb

In [ ]:
xgbparam = {
              'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['binary:logistic'],
              'learning_rate': [0.05], #so called `eta` value
              'max_depth': [3,5,6,7],
              'min_child_weight': [11],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [5,100,250,500], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [1337]
            }

xgbmodel = xgb.XGBClassifier()

xgbGrid_search = GridSearchCV(xgbmodel,xgbparam,cv=5,verbose=0)
xgbres = xgbGrid_search.fit(x_train,y_train)
print(xgbGrid_search.best_params_)

In [ ]:
xgby_pred = xgbGrid_search.predict(x_test)
print("Accuracy score of model Evaluation:  {:.2f}%" .format(accuracy_score(y_test,xgby_pred)*100))
xgbaccscore = xgbGrid_search.best_score_ *100
print("Accuracy for our training dataset with tuning is : {:.2f}%".format(xgbaccscore) )

In [ ]:
xgb_res=pd.DataFrame(xgbGrid_search.cv_results_)
sns.lineplot(data=xgb_res, x='param_max_depth', y='mean_test_score', hue='param_n_estimators')

In [ ]:
print(classification_report(y_test,xgby_pred))

In [ ]:
cm = confusion_matrix(y_test,xgby_pred)
plot_confusion_matrix(xgbGrid_search,x_test,y_test)

In [ ]:
xgy_pred = np.where(xgby_pred>0.5,1,0)
print("ACcuracy Score:",accuracy_score(y_test,xgy_pred)*100)

In [ ]:
xgrmse = sqrt(mean_squared_error(y_test, xgby_pred))
print("Root Mean Squared Error: ", xgrmse*100)

In [ ]:
# pickle.dump(xgbGrid_search.best_estimator_, open("/content/drive/MyDrive/project/model/XGB.pkl", "wb"))

**ANN using KERAS**

**ANN MODEL**

In [ ]:
import tensorflow as tf
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import Sequential,models,layers
from tensorflow.keras.layers import Dense,Activation
from keras_visualizer import visualizer 

from IPython.display import Image

from livelossplot import PlotLossesKeras

**KERAS ANN**

In [ ]:
optim =  tf.keras.optimizers.Adam(learning_rate=0.001)
plotlosses = PlotLossesKeras()

metrics=[
    tf.keras.metrics.BinaryAccuracy(name='acc'),
    tf.keras.metrics.AUC(name='auc')
    ]

def create_model(optimizer=optim, hidden_size=16):
    Kmodel = Sequential()
    Kmodel.add(Dense(units=hidden_size, activation='relu', input_shape=(x_train.shape[1],)))
    Kmodel.add(Dense(units=16, activation='relu'))
    Kmodel.add(Dense(units=1, activation='sigmoid'))
    Kmodel.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=metrics)
    return Kmodel

Kmodel = KerasClassifier(build_fn=create_model)

param_grid = {
              'epochs': [50], 
              'batch_size': [16], 
              'optimizer': ['adam'], 
              'hidden_size': [32],
              'callbacks' : [plotlosses]
              }

krsgrid_search = GridSearchCV(estimator=Kmodel, param_grid=param_grid, cv=10,verbose=1)
krsgrid_search.fit(x_train, y_train)
print("Best parameters: ", krsgrid_search.best_params_)

In [ ]:
print("Best parameters: ", krsgrid_search.best_params_)
print("Best parameters: ", krsgrid_search.best_estimator_)

**BUILD MODEL USING BEST PARAMETER**

In [ ]:
# Build the model using the best parameters
plotlosses = PlotLossesKeras()
best_params = krsgrid_search.best_params_
best_model = create_model(optimizer=best_params['optimizer'], hidden_size=best_params['hidden_size'])
fitmodel = best_model.fit(x_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'],verbose=0,callbacks =[plotlosses])

In [ ]:
# Plot the training and validation accuracy
plt.plot(fitmodel.history['acc'])
plt.plot(fitmodel.history['auc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
best_model.summary()

In [ ]:
visualizer(best_model, format='png', filename='Anngraph', view=True)
Image(filename='Anngraph.png')

PREDICTION EXAMPLE

In [ ]:
# RMSE OF ALL MODEL

print("RMSE OF XGBOOST: {:.2f}%".format(xgrmse*100))
print("RMSE OF Linear Regression: {:.2f}%".format(lrrmse*100))
print("RMSE OF KNN: {:.2f}%".format(rmse*100))

In [ ]:

# Create lists with the 3 values and their labels
values1 = [xgrmse*100]
values2 = [lrrmse*100]
values3 = [rmse*100]
labels = ['XGBOOST', 'LR', 'KNN']

# Create the dataframe
df = pd.DataFrame({labels[0]:values1, labels[1]:values2, labels[2]:values3})

# Create the graph
ax = sns.barplot(data=df)
plt.title('Comparison of 3 RNN')

# set max range of y-axis to 100
plt.ylim(0,100)

for i in ax.containers:
    ax.bar_label(i,)
# Show the graph
plt.show()

**PREDICTION USING XGBOOST**

In [ ]:
data = {
       'age' : [54],
       'workclass' : ['Private'],
       'education' : ['HS-grad'],
       'occupation' : ['Exec-managerial'],
       'sex' : ['Male'],
       'workHour' : [40],
       'Country' : ['United-States']
    }

In [ ]:
testdata = pd.DataFrame(data)
for columns in testdata.columns[testdata.dtypes == 'object']:
  testdata[columns] = le.fit_transform(testdata[columns])

testdata = np.array(testdata)
print(testdata)

In [ ]:
XGBprediction = xgbGrid_search.best_estimator_
prediction = XGBprediction.predict(testdata)
prediction = np.where(prediction>0.5,1,0)
print("ACcuracy Score: {:.2f}%".format(accuracy_score(y_test,prediction)*100))

if prediction == 0:
  apredictionns = 'Below or Equal to 50,0000Rs'
elif prediction == 1:
  prediction = 'Above 50,0000Rs'